# An sample to finetune WizardCoder-15B distributely on SageMaker

In [ ]:
## Update sagemaker python sdk version
!pip install -U sagemaker

In [4]:
import boto3
import sagemaker
from sagemaker import get_execution_role


sess                     = sagemaker.Session()
role                     = get_execution_role()
sagemaker_default_bucket = sess.default_bucket()

account                  = sess.boto_session.client("sts").get_caller_identity()["Account"]
region                   = sess.boto_session.region_name

In [ ]:
%%script bash
rm -rf src
mkdir src
cp s5cmd src/
cd src

git clone https://github.com/nlpxucan/WizardLM.git
cd WizardLM
git reset --hard 46d1ce7dbbb1f987ae5e5915c75f33b89a6a17ab

cd ../
git clone https://github.com/lm-sys/FastChat.git
cd FastChat
git reset --hard 974537efbd82093b45e64d07904efe7728193a52

## Download pretrained model from HuggingFace Hub

To avoid download model from Huggingface hub failure, we download first and push those model files to S3 bucket first.

In [ ]:
!pip install huggingface_hub

In [ ]:
from huggingface_hub import snapshot_download
from pathlib import Path


local_cache_path = Path("./model")
local_cache_path.mkdir(exist_ok=True)

model_name = "WizardLM/WizardCoder-15B-V1.0"

# Only download pytorch checkpoint files
allow_patterns = ["*.json", "*.pt", "*.bin", "*.model", "*.py"]

model_download_path = snapshot_download(
    repo_id=model_name,
    cache_dir=local_cache_path,
    allow_patterns=allow_patterns,
    revision='926ca1b215c4631bc5f8c3e47173381452c23e5c'
)

**Upload model files to S3**

In [ ]:
# Get the model files path
import os
from glob import glob

local_model_path = None

paths = os.walk(r'./model')
for root, dirs, files in paths:
    for file in files:
        if file == 'config.json':
            print(os.path.join(root,file))
            local_model_path = str(os.path.join(root,file))[0:-11]
            print(local_model_path)
if local_model_path == None:
    print("Model download may failed, please check prior step!")

In [ ]:
%%script env sagemaker_default_bucket=$sagemaker_default_bucket local_model_path=$local_model_path bash

chmod +x ./s5cmd
./s5cmd sync ${local_model_path} s3://${sagemaker_default_bucket}/llm/models/wizardcoder/WizardLM/WizardLM-15B/

rm -rf model

## Prepare docker image

In [ ]:
%%writefile Dockerfile
## You should change below region code to the region you used, here sample is use us-west-2
From 763104351884.dkr.ecr.us-west-2.amazonaws.com/huggingface-pytorch-training:1.13.1-transformers4.26.0-gpu-py39-cu117-ubuntu20.04 


ENV LANG=C.UTF-8
ENV PYTHONUNBUFFERED=TRUE
ENV PYTHONDONTWRITEBYTECODE=TRUE

RUN pip3 uninstall -y deepspeed \
    && pip3 install deepspeed==0.10.0 \
    && pip3 install transformers==4.30.1 \
    && pip3 install accelerate==0.21.0

## Make all local GPUs visible
ENV NVIDIA_VISIBLE_DEVICES="all"

In [ ]:
## You should change below region code to the region you used, here sample is use us-west-2
!aws ecr get-login-password --region us-west-2 | docker login --username AWS --password-stdin 763104351884.dkr.ecr.us-west-2.amazonaws.com

**Build image and push to ECR.**

In [6]:
## define repo name, should contain *sagemaker* in the name
repo_name = "sagemaker-wizardcoder-15b-finetune"

In [ ]:
%%script env repo_name=$repo_name bash

#!/usr/bin/env bash

# This script shows how to build the Docker image and push it to ECR to be ready for use
# by SageMaker.

# The argument to this script is the image name. This will be used as the image on the local
# machine and combined with the account and region to form the repository name for ECR.
# The name of our algorithm
algorithm_name=${repo_name}

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

### Generate the deepspeed config

In [ ]:
%%writefile src/ds.json
{
  "bf16": {
    "enabled": true
  },
  "fp16": {
    "enabled": false,
    "auto_cast": false,
    "loss_scale": 0,
    "initial_scale_power": 16,
    "loss_scale_window": 1000,
    "hysteresis": 2,
    "min_loss_scale": 1
  },
  "optimizer": {
    "type": "AdamW",
    "params": {
      "lr": "auto",
      "betas": "auto",
      "eps": "auto",
      "weight_decay": "auto"
    }
  },
  "scheduler": {
    "type": "WarmupLR",
    "params": {
      "warmup_min_lr": "auto",
      "warmup_max_lr": "auto",
      "warmup_num_steps": "auto"
    }
  },
  "zero_optimization": {
    "stage": 3,
    "overlap_comm": true,
    "contiguous_gradients": true,
    "sub_group_size": 1e9,
    "reduce_bucket_size": "auto",
    "stage3_prefetch_bucket_size": "auto",
    "stage3_param_persistence_threshold": "auto",
    "stage3_max_live_parameters": 1e9,
    "stage3_max_reuse_distance": 1e9,
    "stage3_gather_16bit_weights_on_model_save": true
  },
  "gradient_accumulation_steps": "auto",
  "gradient_clipping": "auto",
  "steps_per_print": 2000,
  "train_batch_size": "auto",
  "train_micro_batch_size_per_gpu": "auto",
  "wall_clock_breakdown": false
}

In [ ]:
%%writefile src/WizardLM/WizardCoder/train_mem.py

# Make it more memory efficient by monkey patching the LLaMA model with FlashAttn.

# # Need to call this before importing transformers.
# from llama_flash_attn_monkey_patch import (
#     replace_llama_attn_with_flash_attn,
# )

# replace_llama_attn_with_flash_attn()

from train_wizardcoder import train

if __name__ == "__main__":
    train()

In [10]:
%%writefile ./src/ds-train-dist.sh
#!/bin/bash
CURRENT_HOST="${SM_CURRENT_HOST}"


IFS=',' read -ra hosts_array <<< "${SM_HOSTS}"
NNODES=${#hosts_array[@]}
NODE_RANK=0

for i in "${!hosts_array[@]}"; do
    if [[ "${hosts_array[$i]}" == *${CURRENT_HOST}* ]]; then
        echo "host index：$i"
        NODE_RANK="$i" 
    fi
done
   
    
MASTER_PORT="13579"
export NCCL_SOCKET_IFNAME="eth0"

#Configure the distributed arguments for torch.distributed.launch.
GPUS_PER_NODE="$SM_NUM_GPUS"
DISTRIBUTED_ARGS="--nproc_per_node $GPUS_PER_NODE \
                  --nnodes $NNODES \
                  --node_rank $NODE_RANK \
                  --master_addr $MASTER_ADDR \
                  --master_port $MASTER_PORT"

chmod +x ./s5cmd
./s5cmd sync s3://$MODEL_S3_BUCKET/llm/models/wizardcoder/WizardLM/WizardLM-15B/* /tmp/wizardcoder_pretrain/


DEEPSPEED_OPTS="""
    WizardLM/WizardCoder/src/train_wizardcoder.py 
    --deepspeed ds.json 
    --model_name_or_path "/tmp/wizardcoder_pretrain/" 
    --data_path WizardLM/WizardCoder/data/alpaca_data.json 
    --output_dir "/tmp/wizardcoder_out" 
    --num_train_epochs 1 
    --per_device_train_batch_size 1 
    --per_device_eval_batch_size  1 
    --gradient_accumulation_steps 4 
    --evaluation_strategy "no" 
    --save_strategy "no" 
    --save_steps 2000 
    --save_total_limit 1 
    --learning_rate 2e-5 
    --weight_decay 0. 
    --warmup_ratio 0.03 
    --lr_scheduler_type "cosine" 
    --logging_steps 1 
    --cache_dir '/tmp' 
    --model_max_length 512 
    --gradient_checkpointing True 
    --bf16 True 
    --tf32 True 
    --report_to "none"
"""    

CMD="torchrun ${DISTRIBUTED_ARGS} ${DEEPSPEED_OPTS}"
echo ${CMD}
${CMD} 2>&1 

if [[ "${CURRENT_HOST}" == "${MASTER_ADDR}" ]]; then  
    ./s5cmd sync /tmp/wizardcoder_out s3://$MODEL_S3_BUCKET/llm/models/wizardcoder/output/WizardLM/WizardLM-15B/$(date +%Y-%m-%d-%H-%M-%S)/
fi

Overwriting ./src/ds-train-dist.sh


In [11]:
## The image uri which is build and pushed above
image_uri = "{}.dkr.ecr.{}.amazonaws.com/{}:latest".format(account, region, repo_name)
image_uri

'928808346782.dkr.ecr.us-west-2.amazonaws.com/sagemaker-wizardcoder-15b-finetune:latest'

In [12]:
import time
from sagemaker.estimator import Estimator

environment = {
              'MODEL_S3_BUCKET': sagemaker_default_bucket # The bucket to store pretrained model and fine-tune model
}

base_job_name = 'wizardcoder-15b-finetune'

instance_type = 'ml.p4d.24xlarge'
# instance_type = 'ml.g5.2xlarge'

estimator = Estimator(role=role,
                      entry_point='ds-train-dist.sh',
                      source_dir='./src',
                      base_job_name=base_job_name,
                      instance_count=1,
                      instance_type=instance_type,
                      image_uri=image_uri,
                      environment=environment,
                      disable_profiler=True,
                      debugger_hook_config=False)


# estimator.fit(inputs)

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [ ]:
estimator.fit()

Using provided s3_resource


INFO:sagemaker:Creating training-job with name: wizardcoder-15b-finetune-2023-07-31-15-27-59-503


2023-07-31 15:28:10 Starting - Starting the training job......
2023-07-31 15:28:51 Starting - Preparing the instances for training.....................
2023-07-31 15:32:31 Downloading - Downloading input data...
2023-07-31 15:32:46 Training - Downloading the training image...............
2023-07-31 15:35:27 Training - Training image download completed. Training in progress.......bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-07-31 15:36:27,757 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-07-31 15:36:27,815 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-07-31 15:36:27,825 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-07-31 15:36:27,826 sagemaker_pytorch_container.training INFO     Invoking user training script.
2023-07-31 15:36:29,609 sagemaker-training-toolkit INFO     No N